### Clau-hee chatbot interaction initial test

In [46]:
print('Welcome to the Compliance Challenge!')
print("Let's test your knowledge of our Gift Policy. Answer True or False.")
print("You'll be asked 5 questions.... on our Gift Policy.")
print("One wrong move.. you will be eliminated!")
print("Answer 5 questions in a row to win!, think smart!")
print("Let the game begin!!")

Welcome to the Compliance Challenge!
Let's test your knowledge of our Gift Policy. Answer True or False.
You'll be asked 5 questions.... on our Gift Policy.
One wrong move.. you will be eliminated!
Answer 5 questions in a row to win!, think smart!
Let the game begin!!


## Compliance Questions

In [39]:
questions = [
    {
        "question": "Based on the Gift Policy, it's acceptable to give a small gift in cash to a vendor during holiday season.",
        "answer": False,
        "level": "easy",
        "explanation": "The policy prohibits cash or cash equivalents, regardless of the amount or occasion."
    },

    {
        "question": "A promotional mug worth $10, given at a public product launch event, is allowed under the Gift Policy.",
        "answer": True,
        "level": "easy",
        "explanation": "The gift is reasonable, below the threshold, openly given, and part of a legitimate business event."
    },

    {
       "question": "According to the Gift Policy, giving a gift in secret is acceptable if the value is within the allowed threshold.",
        "answer": False,
        "level": "medium",
        "explanation": "Gifts must be given openly. Secrecy violates the policy even if the value is under the threshold."
    }, 

     {
       "question": "Sponsorships can include providing your organization's products or services as in-kind contributions to a third party, with the expectation of brand association.",
        "answer": True,
        "level": "hard",
        "explanation": "Sponsorships involve in-kind contributions to promote brand association with an event or activity."
    },  

    {
       "question": "Donations are voluntary payments in the form of money, in-kind contributions, or anything of value to a non-profit or charitable organization, with no expectation of receiving a tangible benefit in return.",
       "answer": True,
       "level": "hard",
       "explanation": "Donations are made without expecting any measurable benefit or service in return."
    }
]


    
    

#### Building dialogue logic 

Presents a question  
Easy medium to difficulty   
Returns if the user incorrectly answers  
5 streak win, wins the game!!


def ask_question(q):
    print(q["question"])
    user_input = input("True or False? ").strip().lower()
    
    is_correct = (user_input == "true" and q["answer"] is True) or (user_input == "false" and q["answer"] is False)
    
    if is_correct:
        print("✅ Correct! " + q["explanation"])
        return True
    else:
        print("❌ Incorrect. " + q["explanation"])
        return False

correct_count = 0
history = []


In [ ]:
def ask_question(q):
    print(f"Clau-Hee: {q['question']}")
    user_input = input("Your answer (True/False): ").strip().lower()
    
    is_correct = (user_input == "true" and q["answer"] is True) or (user_input == "false" and q["answer"] is False)
    
    if is_correct:
        print(f"Clau-Hee: ✅ Correct! {q['explanation']}")
        return True
    else:
        print(f"Clau-Hee: ❌ Incorrect. {q['explanation']}")
        return False

correct_count = 0
history = []

## compliance quiz 

In [41]:
available_questions = [q for q in questions if q["level"] == "easy"]

# Preload next levels
available_questions.extend([q for q in questions if q["level"] == "medium"])
available_questions.extend([q for q in questions if q["level"] == "hard"])

for i in range(5):  # Max of 5 rounds
    q = available_questions[i]
    print(f"\nQuestion {i + 1}: {q['level'].capitalize()}")
    result = ask_question(q)
    history.append({"question": q["question"], "result": result})

    if result:
        correct_count += 1
    else:
        print("\nClau-Hee: Game over! Try again next time.")
        break


Question 1: Easy
Clau-Hee: Based on the Gift Policy, it's acceptable to give a small gift in cash to a vendor during holiday season.
Clau-Hee: ✅ Correct! The policy prohibits cash or cash equivalents, regardless of the amount or occasion.

Question 2: Easy
Clau-Hee: A promotional mug worth $10, given at a public product launch event, is allowed under the Gift Policy.
Clau-Hee: ✅ Correct! The gift is reasonable, below the threshold, openly given, and part of a legitimate business event.

Question 3: Medium
Clau-Hee: According to the Gift Policy, giving a gift in secret is acceptable if the value is within the allowed threshold.
Clau-Hee: ✅ Correct! Gifts must be given openly. Secrecy violates the policy even if the value is under the threshold.

Question 4: Hard
Clau-Hee: Sponsorships can include providing your organization's products or services as in-kind contributions to a third party, with the expectation of brand association.
Clau-Hee: ✅ Correct! Sponsorships involve in-kind contr

In [ ]:

if correct_count == 5:
    print("\nClau-Hee: CONGRATULATIONS! You answered all 5 questions correctly.")
    print("Clau-Hee: You are now a Compliance Champion!")




Clau-Hee: CONGRATULATIONS! You answered all 5 questions correctly.
Clau-Hee: You are now a Compliance Champion! You are Ethical!
